In [622]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from collections import OrderedDict
from scipy.interpolate import interp1d

In [623]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [624]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass, LSSTdefaults
from lsst.sims.utils import angularSeparation

In [625]:
#rname = 'R10'
#rname = 'R22'
#rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
#rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
rname = 'R32'
#rname = 'R24'
#rname = 'R41'
#rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [626]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
useDefault = False
try:
    if np.isnan(dd.rtm[rname]):
        useDefault = True
        print('no data yet, use default')
except TypeError:
    pass

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [627]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

In [628]:
#we do not need this cell to calculate m5, but these FWHMeff are the default values we use actually
lsstDefaults = LSSTdefaults()

### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [629]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

{'detector': '/home/bxin/notebooks/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens1',
 'lens2': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens2',
 'lens3': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens3',
 'filters': '/home/bxin/notebooks/syseng_throughputs/components/camera/filters',
 'mirror1': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/home/bxin/notebooks/syseng_throughputs/siteProperties'}

In [630]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

vendor = dd.vendor[rname]
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector0 = st.buildDetector(vendorDir, addLosses) #design QE from this vendor
detector = Bandpass()

In [631]:
for f in filters:
    print(f, lsstDefaults.FWHMeff(f), lsstDefaults.m5(f))

r 0.83 24.43
i 0.8 24.0
u 0.92 23.68
z 0.78 24.45
g 0.87 24.89
y 0.76 22.6


### Butler access to the QE data

In [632]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

#This is no longer needed after tickets/DM-22605
#from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
#mapper = LsstCamMapper()
#lsstcam = mapper.camera

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [633]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframes for this raft

In [634]:
#filterlist = tuple([s for s in filters]+['fS']) # this would be in different order than below
filterlist = ['u', 'g', 'r', 'i', 'z', 'y', 'fS']
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)
Tdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)
Sdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)

In [635]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    key = rname+'_'+dname
    raDeg = {}
    decDeg = {}
    readnoise = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        #print(key, i, raDec)
        if useDefault:
            readnoise[i] = 8.8
        else:
            readnoise[i] = amp.getReadNoise()
    adf[key].loc['raDeg'] = list(OrderedDict(sorted(raDeg.items())).values())
    adf[key].loc['decDeg'] = list(OrderedDict(sorted(decDeg.items())).values())
    adf[key].loc['readnoise'] = list(OrderedDict(sorted(readnoise.items())).values())
        
    #effetive area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = list(radius)
    adf[key].loc['effarea'] = list(np.interp(radius, vr, vv)*np.pi*(M1D/2)**2)

In [636]:
adf

,R32_S00,R32_S01,R32_S02,R32_S10,R32_S11,R32_S12,R32_S20,R32_S21,R32_S22
raDeg,"[-0.3332777777777778, -0.3048333333333334, -0....","[-0.09838888888888889, -0.06994444444444445, -...","[0.13644444444444442, 0.16488888888888886, 0.1...","[-0.33299999999999996, -0.3045555555555556, -0...","[-0.0978888888888889, -0.06944444444444445, -0...","[0.13699999999999998, 0.16544444444444445, 0.1...","[-0.3322777777777778, -0.30383333333333334, -0...","[-0.09733333333333334, -0.0688888888888889, -0...","[0.13744444444444442, 0.16588888888888886, 0.1..."
decDeg,"[0.41611111111111104, 0.41611111111111104, 0.4...","[0.41538888888888886, 0.41538888888888886, 0.4...","[0.4151111111111111, 0.4151111111111111, 0.415...","[0.6509444444444444, 0.6509444444444445, 0.650...","[0.6504999999999999, 0.6505, 0.650499999999999...","[0.6499444444444444, 0.6499444444444444, 0.649...","[0.8858333333333333, 0.8858333333333331, 0.885...","[0.8853888888888889, 0.885388888888889, 0.8853...","[0.8849444444444445, 0.8849444444444445, 0.884..."
radDeg,"[0.533123420145497, 0.51581991448053, 0.499537...","[0.42688187226069907, 0.42123635717621705, 0.4...","[0.4369599469565564, 0.4466599428335051, 0.457...","[0.7311722770299125, 0.7186645708411321, 0.707...","[0.6578237399569877, 0.6541961278210294, 0.651...","[0.6642258469821385, 0.6706701655738866, 0.678...","[0.946097467905741, 0.936487070558382, 0.92764...","[0.8907224642057161, 0.8880646293080864, 0.886...","[0.8955535399207167, 0.9003574872837781, 0.906..."
effarea,"[34.059016026765356, 34.07112609863295, 34.081...","[34.11716353204808, 34.11716353204808, 34.1109...","[34.11716353204808, 34.08921555181896, 34.0419...","[34.03159407800889, 34.04582843233953, 34.0336...","[34.06972369665732, 34.072262525755, 34.073945...","[34.06377014656438, 34.05701378618837, 34.0490...","[33.79045263312475, 33.79076913863542, 33.7969...","[33.88575822738149, 33.88761834591734, 33.8888...","[33.88237713945404, 33.877889664728414, 33.856..."
readnoise,"[5.0598, 5.09675, 5.06342, 5.04945, 5.10136, 5...","[5.02211, 5.10931, 5.02567, 5.07887, 5.0533, 5...","[5.09453, 5.06142, 5.16877, 5.0751, 5.16766, 5...","[5.18321, 5.29641, 5.33045, 5.27752, 5.33982, ...","[4.89794, 4.50032, 4.50876, 4.47149, 4.72429, ...","[4.94637, 4.98654, 4.99473, 4.97253, 4.99978, ...","[4.98501, 5.01925, 5.04659, 5.05025, 5.05474, ...","[5.14015, 5.29515, 5.55189, 5.15327, 5.63362, ...","[5.07681, 5.06373, 5.13081, 5.09981, 5.13085, ..."


In [637]:
ampList = list(OrderedDict(sorted(raDeg.items())).keys())

In [638]:
# this is needed if there are only 6 QE measurements per amp
idx = np.where(detector0.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector0.wavelen[idx1]
y1 = detector0.sb[idx1]
x2 = detector0.wavelen[idx2]
y2 = detector0.sb[idx2]

In [639]:
m5SRD = np.array([23.9, 25.0, 24.7, 24.0, 23.3, 22.1])
#m5SRDmin = []
# Nv1 from SRD table 24
Nv1 = np.array([56, 80, 184, 184, 160, 160])
omega = Nv1/sum(Nv1)
fidx = 'ugrizy' #very important!

In [640]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert useDefault or dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filterlist:
        m5df[key][f] = [-1.]*len(ampList)
    for f in filterlist[:6]:
        Tdf[key][f] = [-1.]*len(ampList)
        Sdf[key][f] = [-1.]*len(ampList)
        
    for amp in det:
        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, calibDate='1970-01-01T00:00:00')
            wavelen = detector0.wavelen 

            k = amp.getName()
            if len(qe_curve.data[k][0])>10:
                wlen = qe_curve.data[k][0]
                eff = qe_curve.data[k][1]
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
            else:
                aa = np.append(x1, qe_curve.data[k][0].value)
                aa = np.append(aa, x2)
                wlen = aa * qe_curve.data[k][0].unit

                aa = np.append(y1, qe_curve.data[k][1].value)
                aa = np.append(aa, y2)
                eff = aa * qe_curve.data[k][1].unit
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
            sb = f(wavelen)*0.01
            #alternatively we could do (only for >10 QE measurements)
            #amp_point = amp.getBBox().getCenter()
            #sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data

            sb[np.isnan(sb)] = 0
            if np.max(sb)>1.5:
                print('These seem too LARGE ', k)
                print(np.max(sb))
                sb = 0
            if np.max(sb)<0.01:
                print('dead channel: %s, max sb = %.2f'%(key, np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
        except NoResults:
            if not useDefault:
                print('No results found for this detector')
            assert useDefault
            
            hardware, system = st.buildHardwareAndSystem(defaultDirs)
            
        #calculate m5      
        iamp = ampList.index(amp.getName())
        effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
        readnoise = adf[key]['readnoise'][iamp]
        
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        for f in filters:
            m5df[key][f][iamp] = m5.m5[f]
            Tdf[key][f][iamp] = m5.Tb[f]
            Sdf[key][f][iamp] = m5.Sb[f]
        m5amp = np.array([m5.m5[f] for f in fidx])
        if np.all(m5amp>0):
            m5df[key]['fS'][iamp] = sum(omega*10**(0.8*(m5amp - m5SRD)))
        

Calculating m5 for R32_S00
Calculating m5 for R32_S01
Calculating m5 for R32_S02
Calculating m5 for R32_S10
Calculating m5 for R32_S11
Calculating m5 for R32_S12
Calculating m5 for R32_S20
Calculating m5 for R32_S21
Calculating m5 for R32_S22


In [652]:
Sdf

,R32_S00,R32_S01,R32_S02,R32_S10,R32_S11,R32_S12,R32_S20,R32_S21,R32_S22
u,"[0.053161976879294824, 0.0537874895802891, 0.0...","[0.05510572133244716, 0.055442564202295645, 0....","[0.056591487125380646, 0.0575667153699095, 0.0...","[0.05341157639598713, 0.053956111789255676, 0....","[0.06269999413028957, 0.062317876940930064, 0....","[0.052247964508510936, 0.05320610886847102, 0....","[0.054206921861525696, 0.05483215672392146, 0....","[0.05358743593576786, 0.054074769460833956, 0....","[0.04983605866612993, 0.05092382654094133, 0.0..."
g,"[0.1557426212756447, 0.15607883784309837, 0.15...","[0.15833702191253665, 0.15874742175818712, 0.1...","[0.15789778102694596, 0.1585280495563198, 0.15...","[0.15642031442438856, 0.15682951960218486, 0.1...","[0.15472502597546287, 0.15436096855718115, 0.1...","[0.15531243437254558, 0.1559758823801542, 0.15...","[0.15675652484623478, 0.15727997871542643, 0.1...","[0.15728751416291487, 0.15736850797950627, 0.1...","[0.15875867084525244, 0.15947538708144554, 0.1..."
r,"[0.12459462261735389, 0.12463736184629688, 0.1...","[0.12569084787775253, 0.12590377479511952, 0.1...","[0.12529190811506347, 0.12543288855746373, 0.1...","[0.12516313056025366, 0.1253069486994434, 0.12...","[0.11697868376016256, 0.1169010233505235, 0.11...","[0.12512909191205793, 0.12526118405163242, 0.1...","[0.12507581523008537, 0.12526676021194016, 0.1...","[0.1257494285972021, 0.12556758428402331, 0.12...","[0.1261642573563609, 0.12636433255019303, 0.12..."
i,"[0.0869064527717668, 0.08706411473803279, 0.08...","[0.08713785804643354, 0.08724995072723937, 0.0...","[0.08670009767071657, 0.0868480845494664, 0.08...","[0.0865300958266675, 0.08673238164043975, 0.08...","[0.08159515672237025, 0.08160451462203953, 0.0...","[0.08727958902647474, 0.08739529490881821, 0.0...","[0.08707212713158602, 0.0872936222846525, 0.08...","[0.08776912834173986, 0.08762042893714106, 0.0...","[0.08605411913556173, 0.08625259172989838, 0.0..."
z,"[0.05569174580450758, 0.0558434158994073, 0.05...","[0.0554754390415226, 0.05553959671721469, 0.05...","[0.05519371891077175, 0.05530354967753047, 0.0...","[0.055079401532202296, 0.05521208341065764, 0....","[0.051581327172960885, 0.05160020874034361, 0....","[0.05574693821978813, 0.055835119908755886, 0....","[0.0557583415570838, 0.05590061428198188, 0.05...","[0.056248174733576614, 0.0561327706824862, 0.0...","[0.05457356450620003, 0.05471234699214429, 0.0..."
y,"[0.030204169135542486, 0.030250954730209507, 0...","[0.029900710440831944, 0.029924728183008065, 0...","[0.029768844398143205, 0.02985898396165752, 0....","[0.029899620645264222, 0.029795255324660503, 0...","[0.02627165798985332, 0.026227958815284093, 0....","[0.03012611575815057, 0.030134468434295954, 0....","[0.03019740778961163, 0.03027558234423767, 0.0...","[0.030484209254820784, 0.030377067199505555, 0...","[0.029558466654545645, 0.02954808476482453, 0...."


In [642]:
m5df

,R32_S00,R32_S01,R32_S02,R32_S10,R32_S11,R32_S12,R32_S20,R32_S21,R32_S22
u,"[24.168758390582777, 24.175078613749548, 24.18...","[24.201006003788567, 24.19875850574673, 24.206...","[24.215378963694366, 24.23042939965119, 24.231...","[24.161656250743288, 24.16065724562879, 24.163...","[24.27797684660632, 24.301611649380067, 24.300...","[24.16474937010637, 24.17532446796524, 24.1843...","[24.183613733144675, 24.18970099820872, 24.193...","[24.16450200279732, 24.158892941825236, 24.148...","[24.11365164197493, 24.13150839110512, 24.1384..."
g,"[24.961898344081945, 24.962727787949213, 24.96...","[24.974080818537146, 24.973837798953273, 24.97...","[24.970856653726656, 24.973630833990004, 24.97...","[24.961250027198698, 24.96067865061705, 24.960...","[24.96313600913585, 24.969916791943962, 24.970...","[24.962516145198467, 24.964274819129155, 24.96...","[24.9626862049771, 24.964076633749738, 24.9651...","[24.963065738842378, 24.9600700995351, 24.9573...","[24.969575197823797, 24.972671933246158, 24.97..."
r,"[24.49131501642733, 24.49132072464275, 24.4914...","[24.49787917097629, 24.49791519574159, 24.4982...","[24.495244430072038, 24.495800560561072, 24.49...","[24.49215044254727, 24.491789112005428, 24.491...","[24.457350571868467, 24.461348761984624, 24.46...","[24.495040723351128, 24.495124557529515, 24.49...","[24.48981734409082, 24.49034655430937, 24.4909...","[24.492850469991993, 24.490298589563835, 24.48...","[24.49572769656115, 24.496725148625632, 24.496..."
i,"[24.01763998057514, 24.018502289432966, 24.018...","[24.020783305441483, 24.020808532017856, 24.02...","[24.017347867230374, 24.018106392041602, 24.01...","[24.01408650902564, 24.014642845124275, 24.014...","[23.983190945078444, 23.986550632254584, 23.98...","[24.021220265392227, 24.02151404306621, 24.021...","[24.0148772560139, 24.01602416353634, 24.01693...","[24.01967799051553, 24.01742731873772, 24.0168...","[24.009686287368957, 24.01100586379048, 24.011..."
z,"[23.404779810290307, 23.406302305876842, 23.40...","[23.40398039512793, 23.40412446808005, 23.4039...","[23.40065986580607, 23.401498595605645, 23.400...","[23.397433171335575, 23.398425276676463, 23.39...","[23.363811384855786, 23.36644224904613, 23.367...","[23.406199977404192, 23.406763119322264, 23.40...","[23.401454106368888, 23.402700763911746, 23.40...","[23.407051921607675, 23.405012441776492, 23.40...","[23.39045311523917, 23.391934086173563, 23.392..."
y,"[22.50567465137244, 22.506582689528546, 22.506...","[22.501144435568534, 22.501217704676485, 22.50...","[22.498356505829694, 22.499733351796415, 22.49...","[22.499010064991836, 22.496782268355595, 22.49...","[22.428479164420196, 22.429419096273513, 22.43...","[22.504788653816718, 22.504664842320913, 22.50...","[22.501439783336217, 22.502745714483105, 22.50...","[22.50764100754501, 22.50501896942604, 22.5050...","[22.490590660706744, 22.490379860082808, 22.48..."
fS,"[1.2301811816292811, 1.2333412753398045, 1.235...","[1.2384948078907543, 1.2381028231173967, 1.240...","[1.2353960311897292, 1.2411507685143897, 1.238...","[1.2191853096578296, 1.2177881482805546, 1.217...","[1.1603725851694424, 1.1716568442604858, 1.173...","[1.231985639924444, 1.2347578767128755, 1.2367...","[1.2271920947278299, 1.230867815097823, 1.2322...","[1.2332788610275287, 1.2271003145357595, 1.223...","[1.201225133341582, 1.2064258274770725, 1.2073..."


In [643]:
dfDir = os.path.join('m5_output', rname)

In [644]:
if not os.path.exists(dfDir):
    os.mkdir(dfDir)
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
adf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
m5df.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Tdf_%s.csv'%rname)
Tdf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Sdf_%s.csv'%rname)
Sdf.to_csv(dfPath)

### Make sure we can read out the dataframes correctly

In [645]:
from ast import literal_eval

In [646]:
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [647]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

raDeg        [-0.3332777777777778, -0.3048333333333334, -0....
decDeg       [0.41611111111111104, 0.41611111111111104, 0.4...
radDeg       [0.533123420145497, 0.51581991448053, 0.499537...
effarea      [34.059016026765356, 34.07112609863295, 34.081...
readnoise    [5.0598, 5.09675, 5.06342, 5.04945, 5.10136, 5...
Name: R32_S00, dtype: object

In [648]:
adf

,R32_S00,R32_S01,R32_S02,R32_S10,R32_S11,R32_S12,R32_S20,R32_S21,R32_S22
raDeg,"[-0.3332777777777778, -0.3048333333333334, -0....","[-0.09838888888888889, -0.06994444444444445, -...","[0.13644444444444442, 0.16488888888888886, 0.1...","[-0.33299999999999996, -0.3045555555555556, -0...","[-0.0978888888888889, -0.06944444444444445, -0...","[0.13699999999999998, 0.16544444444444445, 0.1...","[-0.3322777777777778, -0.30383333333333334, -0...","[-0.09733333333333334, -0.0688888888888889, -0...","[0.13744444444444442, 0.16588888888888886, 0.1..."
decDeg,"[0.41611111111111104, 0.41611111111111104, 0.4...","[0.41538888888888886, 0.41538888888888886, 0.4...","[0.4151111111111111, 0.4151111111111111, 0.415...","[0.6509444444444444, 0.6509444444444445, 0.650...","[0.6504999999999999, 0.6505, 0.650499999999999...","[0.6499444444444444, 0.6499444444444444, 0.649...","[0.8858333333333333, 0.8858333333333331, 0.885...","[0.8853888888888889, 0.885388888888889, 0.8853...","[0.8849444444444445, 0.8849444444444445, 0.884..."
radDeg,"[0.533123420145497, 0.51581991448053, 0.499537...","[0.42688187226069907, 0.42123635717621705, 0.4...","[0.4369599469565564, 0.4466599428335051, 0.457...","[0.7311722770299125, 0.7186645708411321, 0.707...","[0.6578237399569877, 0.6541961278210294, 0.651...","[0.6642258469821385, 0.6706701655738866, 0.678...","[0.946097467905741, 0.936487070558382, 0.92764...","[0.8907224642057161, 0.8880646293080864, 0.886...","[0.8955535399207167, 0.9003574872837781, 0.906..."
effarea,"[34.059016026765356, 34.07112609863295, 34.081...","[34.11716353204808, 34.11716353204808, 34.1109...","[34.11716353204808, 34.08921555181896, 34.0419...","[34.03159407800889, 34.04582843233953, 34.0336...","[34.06972369665732, 34.072262525755, 34.073945...","[34.06377014656438, 34.05701378618837, 34.0490...","[33.79045263312475, 33.79076913863542, 33.7969...","[33.88575822738149, 33.88761834591734, 33.8888...","[33.88237713945404, 33.877889664728414, 33.856..."
readnoise,"[5.0598, 5.09675, 5.06342, 5.04945, 5.10136, 5...","[5.02211, 5.10931, 5.02567, 5.07887, 5.0533, 5...","[5.09453, 5.06142, 5.16877, 5.0751, 5.16766, 5...","[5.18321, 5.29641, 5.33045, 5.27752, 5.33982, ...","[4.89794, 4.50032, 4.50876, 4.47149, 4.72429, ...","[4.94637, 4.98654, 4.99473, 4.97253, 4.99978, ...","[4.98501, 5.01925, 5.04659, 5.05025, 5.05474, ...","[5.14015, 5.29515, 5.55189, 5.15327, 5.63362, ...","[5.07681, 5.06373, 5.13081, 5.09981, 5.13085, ..."


In [649]:
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [650]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

u     [24.168758390582777, 24.175078613749548, 24.18...
g     [24.961898344081945, 24.962727787949213, 24.96...
r     [24.49131501642733, 24.49132072464275, 24.4914...
i     [24.01763998057514, 24.018502289432966, 24.018...
z     [23.404779810290307, 23.406302305876842, 23.40...
y     [22.50567465137244, 22.506582689528546, 22.506...
fS    [1.2301811816292811, 1.2333412753398045, 1.235...
Name: R32_S00, dtype: object

In [651]:
df['R32_S11']['u']

'[24.27797684660632, 24.301611649380067, 24.30038112598368, 24.300679404626976, 24.28390764476417, 24.299787543991993, 24.302711828108585, 24.259800080737925, 24.27311221513383, 24.265832600529386, 24.279000699449973, 24.270146748865876, 24.250769197468102, 24.288053261378575, 24.298403494392552, 24.29774941740135]'